In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import string



In [2]:
datapath = "/Users/aavattikutis/Documents/epidemicmodel/cccruns/fits/fit2/tables/"
models = ["fulllinearmodel_fit_table.csv","reducedlinearmodelNegBinom_fit_table.csv",
          "reducedlinearmodelq0_fit_table.csv","reducedlinearmodelq0ctime_fit_table.csv",
         "nonlinearmodelq0ctime_fit_table.csv","nonlinearmodel_fit_table.csv"]
model_hash = {}
k = -1
for model in models:
    k += 1
    model_hash[model] = string.ascii_uppercase[k]

df = pd.DataFrame.from_dict(model_hash, orient='index')
df.to_csv('../postfit_derivatives/model_hash.csv', header=False)

In [3]:
rois = []
for model in models:
    df = pd.read_csv(datapath + model) #get rois in all tables (some may have failed)
    rois += list(df.roi.unique())

    
rois = list(set(rois))

#get inferred
theta = df.columns[2:] 
ntheta = len(theta)



In [4]:
#plot side by side 

def simpleaxis(ax):
    ax.spines['top'].set_visible(False)
    ax.spines['bottom'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['left'].set_visible(False)
             
roi_us = np.sort([i for i in rois if i[:2]=='US'])#[::-1]
roi_other = np.sort([i for i in rois if i[:2]!='US'])#[::-1]
rois = list(roi_us) + list(roi_other)

dfwaic = pd.DataFrame(columns=['roi', 'model', 'waic'])

k = 0
for roi in rois:
#     print(roi)
    for model in models:
        k += 1
        df = pd.read_csv(datapath + model)
        try:
            measure = df.loc[(df.roi==roi)&(df['quantile']==0.5),'waic'].values[0]
#             dfwaic.loc[k] = [roi,model_hash[model],measure]
            dfwaic.loc[k] = [roi,model.split('.csv')[0],measure]
        except:
#             dfwaic.loc[k] = [roi,model_hash[model],1e6]
            dfwaic.loc[k] = [roi,model.split('.csv')[0],1e6]



In [5]:
dfwaic_sorted = dfwaic.sort_values(['waic'],ascending=True).groupby('roi')
# dfwaic_sorted = dfwaic_sorted.head().reset_index(drop=True)
productname = "roimodel_waicsorted_topmodel_modelname.csv"
dfwaic_sorted.head(1).to_csv('../postfit_derivatives/'+productname,index=False)